## Optimized Model: Rethinking the Pipeline

In the previous notebooks, I implemented multiple models of increasing complexity, ranging from logistic regression to deeper neural networks. While these experiments were valuable for understanding core concepts in machine learning and deep learning, the resulting performance remained relatively low and inconsistent.

After analyzing the results more carefully, it became clear that the limitation was not only related to model architecture or optimization techniques, but also to **how the data was labeled and interpreted**.

### Identified Issue with Data Labeling

In the earlier approach, an image was classified as *damaged* if **any single polygon** within the image was labeled as `D_Building` or `Debris`. This means that even a small, localized damaged region could cause the entire image to be labeled as damaged.  
Such a strategy likely introduces noise and label ambiguity, especially for images that are largely intact but contain minor damage.

This coarse labeling scheme may prevent the model from learning meaningful visual patterns related to *overall structural damage*, which is the core objective of this project.

### Objective of This Notebook

In this notebook, I aim to build the **most optimized model so far**, not only by:
- improving model architecture,
- applying better initialization, regularization, and optimization techniques,

but also by **revisiting and refining the data labeling strategy itself**.

By aligning the labels more closely with the true semantic meaning of structural damage, the goal is to provide the model with cleaner supervision and enable more reliable learning.

This step marks a transition from experimenting with models to **systematically improving the full machine learning pipeline**, from data understanding to final evaluation.


## Parsing the XML file 
the code checks the area of the polygon and if classify it accourding to its portion of the image

In [ ]:
# Put at top of the cell
import os
import xml.etree.ElementTree as ET
from typing import Dict, Tuple, List
import numpy as np
import cv2

def polygon_area(coords: List[Tuple[float, float]]) -> float:
    """Compute polygon area using the shoelace formula.
    coords: list of (x, y) tuples in vertex order (clockwise or ccw).
    """
    if len(coords) < 3:
        return 0.0
    x = np.array([p[0] for p in coords], dtype=float)
    y = np.array([p[1] for p in coords], dtype=float)
    # use roll(-1) to get x_i * y_{i+1}
    return 0.5 * abs(np.dot(x, np.roll(y, -1)) - np.dot(y, np.roll(x, -1)))


def parse_destroyed_with_size_check(path: str, min_coverage: float = 0.05) -> Dict[str, int]:
    """
    Return {basename(filename): 0/1} where 1 means the image contains at least
    one polygon labeled as destroyed and that polygon covers >= min_coverage of image area.

    - path: xml annotation file path
    - min_coverage: fraction of image area (0..1), e.g. 0.05 -> 5%
    """
    DESTROYED_LABELS = {"D_Building", "Debris"}
    result: Dict[str, int] = {}

    try:
        tree = ET.parse(path)
    except ET.ParseError as e:
        raise RuntimeError(f"Failed to parse XML {path}: {e}")
    except FileNotFoundError:
        raise RuntimeError(f"XML file not found: {path}")

    root = tree.getroot()

    for image in root.findall(".//image"):
        filename = image.get("name")
        if not filename:
            continue
        # normalize to basename so it matches files in your images folder
        filename_key = os.path.basename(filename)

        # get image size (some annotations may store width/height as attributes)
        try:
            width = int(image.get("width", 0))
            height = int(image.get("height", 0))
        except ValueError:
            width = 0
            height = 0
        image_area = float(width * height)

        # if size missing, try to read size from a nested tag or skip
        if image_area == 0:
            # fallback: mark as not destroyed (or optionally skip)
            result[filename_key] = 0
            continue

        is_destroyed = False

        for polygon in image.findall("polygon"):
            label = polygon.get("label")
            points = polygon.get("points")

            if not label or not points:
                continue
            if label not in DESTROYED_LABELS:
                continue

            # Flexible parsing of points:
            # common formats: "x1,y1;x2,y2;..." or "x1,y1 x2,y2 ..." or "x1,y1;x2,y2;"
            pts_str = points.strip()
            if ";" in pts_str:
                raw_pts = pts_str.split(";")
            else:
                raw_pts = pts_str.split()  # split on whitespace

            coords = []
            for p in raw_pts:
                p = p.strip()
                if not p:
                    continue
                # support "x,y" or "x,y," etc.
                if "," not in p:
                    # unexpected format
                    coords = []
                    break
                try:
                    x_str, y_str = p.split(",")[:2]
                    coords.append((float(x_str), float(y_str)))
                except Exception:
                    coords = []
                    break

            if not coords:
                continue

            poly_area = polygon_area(coords)
            coverage = poly_area / image_area

            if coverage >= min_coverage:
                is_destroyed = True
                break

        result[filename_key] = int(is_destroyed)

    return result

# Preparing the data 

In [ ]:
def load_and_resize_images(images_folder: str, target_size: Tuple[int, int] = (64, 64)):
    """
    Loads image files, resizes to target_size (width, height), normalizes pixels to [0,1].
    Returns:
      X: numpy array of shape (n_images, height, width, 3)
      ordered_filenames: list of basenames corresponding to rows in X
    """
    X = []
    ordered_filenames = []

    for filename in sorted(os.listdir(images_folder)):
        if filename.lower().endswith((".jpg", ".jpeg", ".png")):
            img_path = os.path.join(images_folder, filename)
            img = cv2.imread(img_path)
            if img is None:
                print(f"Warning: Cannot read {filename}, skipping.")
                continue
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            # cv2.resize expects (width, height) as tuple (w,h) for target size -> (w,h)
            img_resized = cv2.resize(img, target_size)
            img_resized = img_resized.astype(np.float32) / 255.0

            X.append(img_resized)
            ordered_filenames.append(filename)

    X = np.array(X)
    print("Final X shape:", X.shape)
    return X, ordered_filenames

In [ ]:

def build_label_array(ordered_filenames: List[str], labels_dict: Dict[str, int], default_value: int = 0) -> np.ndarray:
    """
    Build label array matching ordered_filenames. Returns shape (n_samples,) of dtype int.
    """
    Y = []
    for fname in ordered_filenames:
        # ensures we compare basenames
        key = os.path.basename(fname)
        if key in labels_dict:
            Y.append(labels_dict[key])
        else:
            print(f"Warning: No label found for {fname}, assigning {default_value}")
            Y.append(default_value)
    return np.array(Y, dtype=int)


In [ ]:
labels_train = parse_destroyed_with_size_check("../EIDSeg_Dataset/data/train/train.xml", min_coverage=0.05)
labels_test  = parse_destroyed_with_size_check("../EIDSeg_Dataset/data/test/test.xml",  min_coverage=0.05)

X_train_org, ordered_filenames_train = load_and_resize_images("../EIDSeg_Dataset/data/train/images/default", target_size=(64,64))
X_test_org,  ordered_filenames_test  = load_and_resize_images("../EIDSeg_Dataset/data/test/images/default",  target_size=(64,64))

Y_train_org = build_label_array(ordered_filenames_train, labels_train)   # shape (n_train,)
Y_test_org  = build_label_array(ordered_filenames_test,  labels_test)    # shape (n_test,)

# quick sanity checks
print("Train positive ratio:", Y_train_org.mean(), "n_train:", Y_train_org.shape[0])
print("Test  positive ratio:", Y_test_org.mean(),  "n_test:",  Y_test_org.shape[0])